# Jump frame range conversion

Converts manually annotated jump intervals from 3x review indices into adjusted 1x frame ranges with fixed padding, then writes a timestamped CSV used for optional jump-window exclusion in movement analysis.

### Analysis steps
1. Parse per-animal range annotations, apply index offset correction, convert 3x indices to 1x frame numbers, add `-3/+3` padding, and export normalized jump ranges.


### Step 1: Convert and export jump ranges

Parses annotated `start-end` ranges per animal/day, applies 3x-to-1x conversion and boundary padding, and writes normalized jump-frame rows to a timestamped CSV file.


In [2]:
# Convert jumping animal frame ranges from 3x speed to 1x, using (start3x-1, end3x-1) for calculations

import pandas as pd
import re
import datetime

TIMESTAMP = datetime.datetime.now().strftime("%Y-%m-%d")

data = [
    ["CK314", "PD00", "-"],
    ["CK314", "PD03", "-"],
    ["CK314", "PD05", "-"],
    ["CK314", "PD11", "-"],
    ["CK315", "PD00", "1536-1541"],
    ["CK315", "PD03", "4941-4946,<br>5479-5484,<br>5862-5867,<br>6279-6283<br>"],
    ["CK315", "PD05", "-"],
    ["CK315", "PD11", "-"],
    ["CK321", "PD00", "-"],
    ["CK321", "PD03", "-"],
    ["CK321", "PD05", "-"],
    ["CK321", "PD11", "-"],
    ["CK324", "PD00", "-"],
    ["CK324", "PD03", "-"],
    ["CK324", "PD05", "-"],
    ["CK324", "PD11", "-"],
    ["CK331", "PD00", "-"],
    ["CK331", "PD03", "-"],
    ["CK331", "PD05", "104-109"],
    ["CK335", "PD00", "2339-2344,<br>2418-2422,<br>2695-2700,<br>2709-2713,<br>2839-2846,<br>3093-3099,<br>4248-4253,<br>5059-5064"],
    ["CK335", "PD03", "1635-1640,<br>2243-2248,<br>2628-2633,<br>3376-3383,<br>3532-3538,<br>3546-3552,<br>3991-3996,<br>4117-4122,<br>4342-4347,<br>4767-4773,<br>4956-4963,<br>5930-5936"],
    ["CK335", "PD05", "-"],
    ["CK335", "PD11", "-"],
    ["CK344", "PD00", "-"],
    ["CK344", "PD03", "-"],
    ["CK344", "PD05", "1910-1916,<br>2059-2064,<br>2370-2378,<br>2449-2453,<br>2535-2541,<br>2624-2628,<br>2741-2747,<br>2820-2827,<br>2915-2921,<br>2996-3005,<br>3024-3031,<br>3075-3081,<br>3090-3097,<br>3256-3262,<br>3320-3325,<br>3392-3399,<br>3406-3413,<br>3471-3476,<br>3482-3488,<br>3547-3553,<br>3563-3570,<br>3578-3585,<br>3684-3688,<br>3748-3756,<br>3830-3835,<br>3848-3853,<br>3939-3943,<br>3948-3951,<br>3959-3964,<br>4034-4038,<br>4049-4053,<br>4150-4154,<br>4159-4166,<br>4234-4238,<br>4248-4252,<br>4352-4356,<br>4363-4368,<br>4465-4469,<br>4478-4483,<br>4493-4500,<br>4568-4573,<br>4581-4586,<br>4631-4635,<br>4643-4647,<br>4668-4675,<br>4757-4761,<br>5023-5027,<br>5034-4039,<br>5048-5053,<br>5061-5066,<br>5126-5130,<br>5135-5141,<br>5150-5156,<br>5715-5720,<br>5724-5729,<br>5808-5812,<br>6412-6414"],
    ["CK345", "PD00", "-"],
    ["CK345", "PD03", "-"],
    ["CK345", "PD05", "-"],
    ["CK345", "PD11", "-"],
]

# Build normalized rows across all animals/days for CSV export.
rows = []
for animal, day, jumps in data:
    if jumps.strip() == "-" or jumps.strip() == "":
        rows.append([animal, day, "", "", "", "", "", ""])
    else:
        ranges = re.split(r',<br>|\s*,\s*', jumps.strip().replace('<br>', ''))
        for rng in ranges:
            if '-' in rng:
                try:
                    start3x_raw, end3x_raw = map(int, rng.strip().split('-'))
                    # Align with original indexing offset from the annotation workflow.
                    start3x = start3x_raw - 1
                    end3x = end3x_raw - 1
                    # Convert reviewed 3x frame references back to original 1x timeline.
                    start1x = start3x * 3
                    end1x = end3x * 3
                    # Add a small buffer window around jumps for conservative exclusion.
                    adj_start1x = start1x - 3
                    adj_end1x = end1x + 3
                    rows.append([animal, day, start3x, end3x, start1x, end1x, adj_start1x, adj_end1x])
                except:
                    continue

jumps_df = pd.DataFrame(rows, columns=[
    "Animal", "Day", "frame_start_3x", "frame_end_3x",
    "frame_start_1x", "frame_end_1x",
    "frame_start_adjusted", "frame_end_adjusted"
])

jumps_df.to_csv(f"D:/CK3_open_field/openfield_analysis_mask_2025-03-31/jump_frames_{TIMESTAMP}.csv", index=False)
